In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch; import torch.nn as nn; import torch.nn.functional as F

## 7.1 Error Surfaces

Consider a change in the weights from $\bf w$ to $\mathbf{w} + \delta \mathbf{w}$. Such a change, will correspond to a change in the error function $E(\bf w)$ that is proportional to its gradient with respect to $\bf w$. That is:
$$\delta E \simeq \delta \mathbf{w}^\intercal \nabla E(\bf w)$$

For smooth and convex $E(\bf w)$, minima will occur where: $$ \nabla E(\mathbf{w}) = \mathbf{0}$$
In principal then, we aim to find minima by iteratively scaling the parameters (e.g. weights) in the direction of $-\nabla E(\bf w)$\
 Well, really we may reach a minima, maxima, or saddle point at points where the gradient vanishes. And indeed, we are typically concerned with high dimensional spaces and error functions with highly nonlinear dependencies on network parameters, so it will often be the case that many minima, maxima, and saddle points exist. Moreover, for any given minima we may generally find many equivalent minima within the parameter space.

While we may rarely be able to hope to find the global minimum, we can get very good results by simply finding sufficient minima.

### Local Quadratic Approximation

This section motivates gradient descent by discussing an approximation to the Newton-Raphson optimization algorithm which I've written about [here](https://github.com/BenAF002/data_science/blob/main/Notes/maths_notes/Newton_optimizer.ipynb)

Consider a point in the weight space $\hat{\bf w}$. The Second-Order Taylor Series expansion (recall that Newton-Raphson only uses expansions of second-order) of $E(\bf w)$ around this point is:
$$E(\mathbf{w}) \simeq E(\hat{\mathbf{w}}) + \bf (w - \hat{w})^\intercal b + \frac{1}{2}(w - \hat{w})^\intercal H(w - \hat{\mathbf{w}})$$
Where $\bf b$ is defined as the gradient of $E$ w.r.t. $\bf w$ evaluated at $\hat{\bf w}$ and $\bf H$ is the Hessian matrix:
$$\mathbf{b} \equiv \nabla E|_{\mathbf{w} = \hat{\mathbf{w}}} \\  \\ \mathbf{H}(\hat{\mathbf{w}}) = \nabla \nabla E(\bf w)|_{w=\hat{w}}$$

The local approximation of the gradient from the Taylor Series expansion is:
$$\nabla E(\bf w) = b + H(w - \hat{w})$$

Let $\bf w^*$ denote the weight values at the minimum of the error function. The term $\bf b$ vanishes from the local quadratic approximation around $\bf w^*$ because the gradient of the error function is 0 at $\bf w^*$. So, the expression for the approximation at this point becomes:
$$E(\mathbf{w}) = E(\mathbf{w}^*) + \frac{1}{2}(\bf w - w^*)^\intercal H(w - w^*)$$
For $\bf H$ evaluated at $\bf w^*$.

Let $\mathbf{u}_i$ denote the $i$-th orthonormal eigenvector of $\bf H$ and $\lambda_i$ denote the associated $i$-th eigenvalue. The eigenvalue equation for $\bf H$ is then:
$$\mathbf{H}\mathbf{u}_i = \lambda_i\mathbf{u}_i \ \rightarrow \ \mathbf{H} = \sum_i \lambda_i \mathbf{u}_i \mathbf{u}_i^\intercal$$
The weights lie within the same vector space spanned by $\bf H$ so they may be expressed as a linear combination of the eigenvectors of $\bf H$. Thus we may use the following expression:
$$\mathbf{w} - \mathbf{w}^* = \sum_i \alpha_i \mathbf{u}_i$$
So, we can rewrite the quadratic approximation at $\bf w^*$ as:
$$E(\mathbf{w}) = E(\mathbf{w}^*) + \frac{1}{2}\bigg[ \big(\sum_i \alpha_i \mathbf{u}_i^\intercal \big) \sum_i \lambda_i \mathbf{u}_i \mathbf{u} \big(\sum_i \alpha_i \mathbf{u}_i \big) \bigg] \\ \ \\ = E(\mathbf{w}^*) + \frac{1}{2}\sum_i \lambda_i \alpha_i^2$$

*Note*: This is easily demonstrated by paper for the 2-dimensional case of $i=[1, 2]$

We may observe from this expression that the error function $E(\bf w)$ will be greater than $E(\mathbf{w}^*)$ if all of the eigenvalues of the Hessian matrix $\bf H$ are positive. This indicates a powerful finding:
- A point in weight space $\bf w^*$ at which $\nabla E(\bf w^*) = 0$ is:
    - A minimum if all of the eigenvalues of the Hessian evaluated at $\bf w^*$ are positive
    - A maximum if all of the eigenvalues of the Hessian evaluated at $\bf w^*$ are negative
    - A saddle point if all of the eigenvalues of the Hessian evaluated at $\bf w^*$ are mixed

More concretely:
> A necessary and sufficient condition for $\bf w^*$ to be a local minimum is that $\nabla E(\bf w) = 0$ *and* the Hessian $\bf H$ is positive definite (i.e. $\bf x^\intercal Hx = 0, \ \forall x$ or equivalently, all of the eigenvalues of $\bf H$ are positive)

**Aside**:\
The fact that we may determine positive definiteness from the eigenvalues of the Hessian is deducible as follows:
- The Hessian $\bf H$ is a square, symmetric matrix; thus it always has real eigenvalues and a complete set of eigenvectors $\{\mathbf{u}_i\}$
- Because the eigenvectors of the Hessian form a complete set, they may represent any arbitrary vector $\bf v$ in the vector space spanned by the Hessian as:
$$\mathbf{v} = \sum_i c_i \mathbf{u}_i$$
- $\bf H$ is positive definite if and only if $\bf v^\intercal H v > 0, \ \ \forall v$
    - Equivalently, if and only if $\mathbf{v}^\intercal \mathbf{H} \mathbf{v} = \sum_i c_i^2 \lambda_i > 0, \ \ \forall \lambda_i$
So, if all of the eigenvalues of $\bf H$, $\lambda_i$ are positive, then the Hessian is positive definite.

## 7.2 Gradient Descent Optimization

The basic approach:
$$\mathbf{w}^{(\tau)} = \mathbf{w}^{(\tau - 1)} + \Delta\mathbf{w}^{(\tau - 1)}$$

There is a great deal of nuance involved in both the selection of the weight vector update $\Delta\mathbf{w}^{(\tau)}$ and the weight initializations $\mathbf{w}^{(0)}$, as both of these things can have a very large impact on the solution found.

### Batch Gradient Descent

The simplest approach to updating with gradient information is to choose the weight update such that there is a small step in the direction of the negative gradient (of the error function w.r.t. the parameters).
$$\mathbf{w}^{(\tau)} = \mathbf{w}^{(\tau - 1)} - \eta \nabla E(\mathbf{w}^{(\tau - 1)})$$
Where $\eta > 0$ is tunable hyperparameter called the *learning rate*.

Typically, we define the error function over the entire training set (e.g., often we sum over $N$). So, this approach to iterative parameter updating requires evaluating with the entire training dataset. Techniques that use the whole data set at once are called ***Batch Methods***.\
(This is a little annoying bc I usually think of "batches" as minibatches and not the full training dataset)...

### Stochastic Gradient Descent

Using the entire training dataset at once can be very inefficient when the training dataset is large. So, we can improve efficiency by splitting up the dataset into *minibatches* and training over those instead. At the most granular level, we could have $N$ minibatches, each of size 1, such that each individual observation is treated as a minibatch. Then,
$$E(\mathbf{w}) = \sum_{n=1}^NE_n(\mathbf{w}) \\ \\ \mathbf{w}^{(\tau)} = \mathbf{w}^{(\tau -1)} - \eta \nabla E_n(\mathbf{w}^{(\tau - 1)})$$

This is the crux of SGD. An ***Epoch*** is then a complete pass through the training data.

There are other benefits to SGD besides reducing computational complexity at each iteration. One is that it reduces the risk of getting stuck on a poor local minimum or saddle point because stationary points w.r.t. the entire training dataset will generally not be stationary points w.r.t. smaller subsets of the training data. Another is that it can help improve the speed of parameter optimization. Consider the gradient of the error function w.r.t. the entire training dataset. The partial derivatives of each parameter (e.g. each weight in $\bf w$) represents a sort of average relationship between that parameter and the error function at every input value. So, different relationships that may be observed at different points in the input space become muted or offset one-another in the gradient. Thus, the update at each step does not change the parameter value by as much as it otherwise could when using smaller minibatches (I think)...

### Mini-Batches

This second benefit that I noted can prove to be a downside when we train with very few observations at each iteration. The gradient of the error function computed from a single data point is a very noisy estimate of the gradient computed on the full data set, and too much noise can cause parameters to vary too much. An intermediate approach is to use *mini-batches* of size greater than 1. 

### Parameter Initialization

The solution obtained through gradient descent is heavily dependent upon parameter initialization.

One key consideration is the concept of *symmetry breaking*. We don't want parameters that are constant (e.g. all initialized to 0), because then they will all comput the same output values and be completely redundant. Similarly, we don't want systemic trends in the parameter initializations for the same reason - i.e. to avoid redundant parameters that *arbitrarily* produce similar outputs and therefore *arbitrarily* move together when updated through gradient descent.

So, we want to initialize parameters randomly.

Additionally, if we are using ReLU activations, then we should be careful to ensure that most initial pre-activations (at least in the early layers of the network) are positive so that we don't prematurely kill neurons. One way to do this without systematically biasing the weight initializations is to initialize the bias parameters as small positive values.

## 7.3 Convergence

At a high-level, gradient descent is a very inefficient process. In order to precent instability and divergence, we typically need the learning rate $\eta$ to be low, making each iteration of training contribute very little to the final parameter estimate and requiring a high number of iterations.

Consider again the local quadratic approximation *in the neighborhood of the minimum*. The gradient of the error function in this approximation may be expressed as:
$$\nabla E = \sum_i \alpha_i \lambda_i \mathbf{u}_i$$
This follows from the earlier expression $\nabla E(\mathbf{w}) = \mathbf{b} + \bf H(w - \hat{w})$ and is again easily demonstrated by pen and paper for the 2-dimensional case of $i = [1,2]$.

Now, we may express the change in weights as a change in the coefficients $\{\alpha_i\}$:
$$\Delta \mathbf{w} = \sum_i \Delta \alpha_i \mathbf{u}_i$$
Combining this with the general form of gradient descent, we may observe:
$$\Delta \mathbf{w} = \mathbf{w}^{(\tau)} - \mathbf{w}^{(\tau - 1)} = \mathbf{w}^{(\tau - 1)} - \eta \nabla E(\mathbf{w}^{(\tau -1)}) - \mathbf{w}^{(\tau - 1)} = - \eta \nabla E(\mathbf{w}^{(\tau - 1)}) \\ \ \\ = \sum_i \Delta \alpha_i \mathbf{u}_i = - \eta \sum_i \alpha_i \lambda_i \mathbf{u}_i \\ \ \\ \implies \Delta \alpha_i = -\eta\lambda_i \alpha_i$$

Therefore, $$\alpha_i^{\text{new}} = (1 - \eta\lambda_i)\alpha_i^{\text{old}}$$

Now, because the eigenvectors $\{\mathbf{u}_i\}$ of the Hessian $\bf H$ are orthonormal, we may express the coefficient $\alpha_i$ as:
$$\mathbf{u}_i^\intercal (\mathbf{w} - \mathbf{w}^*) = \alpha_i$$
Therefore, one way of interpreting $\alpha_i$ is as the distance (in the weights space) to the minimum along the direction $\mathbf{u}_i$

Now, for $|1  - \eta\lambda_i| <  1$: $$\lim_{T\to\infty} (1-\eta\lambda_i)^{(T)}\alpha_i^{(0)} = \alpha_i^{(T)} = 0$$

Thus, gradient descent will lead to *linear convergence* in the neighborhood of the minimum. Note that if $\eta$ is too small, then convergence will take a very long time. However, when $\eta$ is large such that $|1-\eta\lambda_i|>1$, $\alpha_I^{(T)}$ will diverge at the limit. So, $\eta$ is constrained to $\eta < 2/\lambda_{\max}$ to prevent divergence. Naturally, the rate of convergence is governed by the eigenvalues as well. Specifically, it is limited by the smallest eigenvalue. So, when we set $\eta$ to its largest permissable value, the convergence along the direction corresponding to the smallest eigenvalue is governed by:
$$1 - \frac{2\lambda_{\min}}{\lambda_{\max}}$$
So convergence will take a very long time when the ratio of the smallest eigenvalue to the largest eigenvalue is ver small.

### Momentum

We may account for slow convergence caused by large differences in eigenvalues of the Hessian by adding a *momentum* term to the gradient descent formula:
$$\Delta \mathbf{w}^{(\tau - 1)} = -\eta\nabla E(\mathbf{w}^{(\tau - 1)}) + \mu \Delta \mathbf{w}^{(\tau - 2)}$$
The *momemntum parameter* is $\mu$

Picture a sufficiently small, localized region of the error surface such that $\nabla E$ is approximately constant. In this region, a successive series of updates may be expressed as:
$$\Delta \mathbf{w} = -\eta \nabla E\{1 + \mu + \mu^2 + ...\} \\ \ \\= -\frac{\eta}{1-\mu}\nabla E$$
So, for $\eta < 1$, the momentum term effectively increases the learning rate to $\eta / (1 - \mu)$.

Now, in regions of the error surface that have higher curvature (such that $\nabla E$ is not constant), the successive values of $\Delta \mathbf{w}^{(\tau)}$ will be irratic and often oscillatory. When the successive value *are* oscillatory (as is often the case), then the successive contributions from the momentum term, $\mu \Delta \mathbf{w}^{(\tau - 2)}$, will cancel out, preventing it from causing divergence.

### Learning Rate Schedule

We've seen that too-small values of $\eta$ can retard convergence but too-high values can cause divergence and instability. If we could use a larger $\eta$ earlier in training and a smaller $\eta$ later in training, then we may be able to enjoy the benefits of quicker progress to convergence (i.e. quicker learning) while avoiding the risk of divergence or otherwise *missing* the minimum of the error function. This intuition is the motivation behind a learning rate schedule, wherein $\eta$ is a decreasing function of the training step index $\tau$:
$$\mathbf{w}^{(\tau)} = \mathbf{w}^{(\tau -1)} - \eta^{(\tau-1)} \nabla E_n(\mathbf{w}^{(\tau-1)})$$

There are many ways to specify the learning rate schedule (i.e. function) of $\eta^{(\tau)}$ w.r.t. $\tau$. Examples include linear, power law, or exponential decay.

### RMSProp and Adam

So, we've seen that the optimal learning rate depends on the local curvature of the error surface (in the parameter space) and that this curvature can vary across the directions of the parameter sapce. Adaptive Gradient descent (AdaGrad) aims to improve learning by using *different* learning rates for *each* parameter in the network, such that the distinct learning rates may be tuned in accordance with the curvature and direction of descent along the error surface. The fundamental ingredient in AdaGrad is to scale each learning rate parameter down over the course of training by the accumulated sum of squares of the derivatives computed for that parameter. The scaling parameter is then:
$$r_i^{(\tau)} = r_i^{(\tau - i)} + \bigg(\frac{\partial E(\mathbf{w})}{\partial w_i} \bigg)^2$$
Note that this is a rolling sum of the squared partial derivatives of the error function w.r.t. the parameter $w_i$.\
Then the update formula is:
$$w_i^{(\tau)} = w_i^{(\tau - 1)} - \frac{\eta}{\sqrt{r_i^\tau}+\epsilon}\bigg(\frac{\partial E(\mathbf{w})}{\partial w_i} \bigg)$$
Where $\epsilon$ is a small constant added for numerical stability. Note that the effective learning rate for the parameter $w_i$ is uniquely determined for $i$ as $\eta / (\sqrt{r_i^\tau} + \epsilon)$. Thus, if the partial derivatives of the error funciton w.r.t. $w_i$ are consistently large (in magnitude) then $r_i^{(\tau)}$ will be large and the learning rate will deflated. Conversely, if the partial derivatives are consistently small, then the learning rate will be inflated (or at least less deflated). So, this adjustment should smooth out the learning rate of highly influential parameters somewhat.

So long as $\partial E(\mathbf{w}) / \partial w_i \ne 0$, the learning rate under AdaGrad will approach $0$ as $\tau \to \infty$. This is a problem because we often need a very large number of epochs before finding $\nabla E = 0$, so the AdaGrad procedure can effectively end training prematurely for long training jobs. ***RMSProp*** addresses this problem by replacing the sum of squared partial derivatives with an *exponentially weighted average*:
$$r_i^{(\tau)} = \beta r_i^{(\tau - 1)} + (1 - \beta) \bigg( \frac{\partial E (\mathbf{w})}{\partial w_i} \bigg)^2, \ \ 0 < \beta < 1$$
A typical value is $\beta = 0.9$

***Adam***:\
Adam (standing for "adaptive moments") combines RMSProp with momentum. Adam includes two parameter-specific learning rate modifiers:
$$
s_i^{(\tau)} = \beta_1 s_i^{(\tau-1)} + (1 - \beta_1) \bigg(\frac{\partial E(\mathbf{w})}{\partial w_i} \bigg) \\ \ \\
r_i^{(\tau)} = \beta_2 r_i^{(\tau-1)} + (1 - \beta_2) \bigg(\frac{\partial E(\mathbf{w})}{\partial w_i} \bigg)^2
$$
These values are then modified as follows to correct for a bias that is introduced when they are initialized as 0:
$$\hat{s_i}^{(\tau)} = \frac{s_i^{(\tau)}}{1 - \beta_1^\tau} \\ \ \\ \hat{r_i}^{(\tau)} = \frac{r_i^{(\tau)}}{1 - \beta_2^\tau}$$

Note that the factor $1/(1-\beta^\tau)$ approaches $1$ as $\tau$ increases because $\beta < 1$. So, when the number of epochs is large, this bias correction may sometimes be omitted. Finally, the update formula is:
$$w_i^{(\tau)} = w_i^{(\tau - 1)} - \eta \frac{\hat{s_i}^{(\tau)}}{\sqrt{\hat{r_i}^{(\tau)}} + \epsilon}$$

## 7.4 Normalization

Normalization adjusts for extreme parameter values. Very large parameter values can lead to exploding gradients which may make convergence challenging or impossible, while very small parameter values may slow down training or prevent gradients descent through parts of the network altogether (vanishing gradients or "dead neurons"). The effects of extrema are largely dependent upon the choices of activation functions in the network as different activations have different asymptotic behavior. 

A common example is ***Saturation***. Saturation occurs when using activations that map to a restrictive range. Large parameter values will produce actiavtions that asymptotically approach the the limits of the range which can cause significant problems, particularly when the range is $(-1, 1)$, as is the case with the *sigmoid* and *tanh* activation functions.

Let's consider these activations:
$$
\text{sigmoid}(a) = \sigma(a) = \frac{1}{1 + e^{-a}} \\ \ \\
\frac{\partial}{\partial a} \sigma(a) = \sigma(a) \cdot (1 - \sigma(a)) \\ \ \\
\text{tanh}(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}} = 2\sigma(2a) - 1 \\ \ \\ 
\frac{\partial}{\partial a} \text{tanh}(a) = 1 - (2\sigma(2a) - 1)^2
$$


The behavior of the gradients at the extrema is then:
$$
\sigma(a) \to 1 \ \ \implies \ \ \partial\sigma(a)/ \partial a \to 0 \\ \ \\
|\text{tanh}(a)| \to 1 \ \ \implies \ \ \partial \text{tanh}(a) / \partial a \to 0
$$
So, the extrema cause gradients to vanish under both activation functions.

### Data Normalization

The first normalization technique is to simply perform some normalization over the input data. We may have data that has vastly different scales across variables. For such datasets, the changes in one set of weights associated with one input variable may have a radically different effect on the output (and thus the error function) than changes in a set of weights associated with a different input variable.

We may mitigate this by simply applying the normal standardization of centering ans scaling the input variables by their means and standard deviations:
$$\tilde{x}_{ni} = \frac{x_{ni} - \mu_i}{\sigma_i}$$
This ensures that the standardized variables have zero-means and unit variances.

One challeng with this approach is that it requires us to re-use the same values of $\mu_i$ and $\sigma_i$ on *all* training, validation, and test data to ensure that all inputs are scaled the same way. Likewise, (I should think) we need to standardize inputs with these same values during inference, which may not be practically appropriate...

### Batch Normalization

Good old batch norm

Batch normalization standardizes the preactivations of each layer (practically, we could standardize the activations instead) over the elements included in a training batch. So, for a mini-batch of size $K$:
$$\mu_i = \frac{1}{K}\sum_{n=1}^K a_{ni} \\ \ \\
\sigma_i^2 = \frac{1}{K}\sum_{n=1}^K (a_{ni} - \mu_i)^2 \\ \ \\
\hat{a}_{ni} = \frac{a_{ni} - \mu_i}{\sqrt{\sigma_i^2 + \epsilon}}
$$
Where $\epsilon$ is a small constant added for numerical stability.

After normalizing, we introduce scale and shift parameters:
$$\tilde{a}_{ni} = \gamma_i \hat{a}_{ni} + \beta_i$$
These are learnable parameters which provide a trainable mean ($\beta_i$) and variance ($\gamma_i^2$) for the mini-batch.

Additionally, we keep track of moving averages of the mini-batch means and standard deviations so that we can apply normalizations during inference:
$$
\bar{\mu}_i^{(\tau)} = \alpha \bar{\mu}_i^{(\tau - 1)} + (1 - \alpha) \mu_i \\
\bar{\sigma}_i^{(\tau)} = \alpha \bar{\sigma}_i^{(\tau - 1)} + (1 - \alpha) \sigma_i
$$

### Layer Normalization

Layer norm is cleaner because it applies normalization over the hidden units of each layer instead of over the elements of the mini-batch:
$$
\mu_n = \frac{1}{M} \sum_{i=1}^M a_{ni} \\ \ \\
\sigma_n^2 = \frac{1}{M} \sum_{i=1}^M (a_{ni} - \mu_i)^2 \\ \ \\
\hat{a}_{ni} = \frac{a_{ni} - \mu_n}{\sqrt{\sigma_n^2 + \epsilon}}
$$
Where $M$ is the number of hidden units in the layer.

With layer norm, we still apply learnable scale and shift parameters for *each hidden unit*, but we no longer need to keep track of moving averages because the same normalization function may be applied during training and inference.
$$\tilde{a}_{ni} = \gamma_i \hat{a}_{ni} + \beta_i$$